In [10]:
from numpy.random import seed
seed(1)

In [11]:
# Dependencies
import numpy as np
import pandas as pd
import tensorflow
tensorflow.keras.__version__

'2.4.0'

In [12]:
import os

In [13]:
activity = pd.read_csv(os.path.join('andrew_data2.csv'))
activity.head()

,id,type,type_m,x_gear_name,start_date_local,x_start_h,name,x_min,x_km,x_min/km,...,start_longitude,timezone,total_photo_count,trainer,upload_id,upload_id_str,utc_offset,x_date,x_elev_%,x_url
0,2850013230,Run,Hike,NaN,07.11.2019 16:46:32,16.8,Afternoon Run,42.1,0.458,92.08,...,NaN,(GMT-08:00) America/Los_Angeles,0,1,3022102505,3022102505,-28800,2019-11-07,NaN,https://www.strava.com/activities/2850013230
1,2895327719,Run,Hike,NaN,22.11.2019 17:05:04,17.1,Afternoon Run,20.1,0.234,85.91,...,NaN,(GMT-08:00) America/Los_Angeles,0,1,3077242534,3077242534,-28800,2019-11-22,NaN,https://www.strava.com/activities/2895327719
2,2832492215,Run,Hike,NaN,31.10.2019 15:41:16,15.7,Afternoon Run,62.1,0.870,71.53,...,NaN,(GMT-08:00) America/Los_Angeles,0,1,3002283821,3002283821,-25200,2019-10-31,NaN,https://www.strava.com/activities/2832492215
3,2832492192,Run,Hike,NaN,28.10.2019 15:17:05,15.3,Afternoon Run,42.1,0.686,61.27,...,NaN,(GMT-08:00) America/Los_Angeles,0,1,3002283809,3002283809,-25200,2019-10-28,NaN,https://www.strava.com/activities/2832492192
4,2877878037,Run,Hike,NaN,18.11.2019 16:00:16,16.0,Afternoon Run,52.1,0.907,57.47,...,NaN,(GMT-08:00) America/Los_Angeles,0,1,3055046850,3055046850,-28800,2019-11-18,NaN,https://www.strava.com/activities/2877878037


In [14]:
activity_clean = activity[["type_m", "x_start_h", "x_min", "average_heartrate", "max_heartrate", "average_cadence"]]
activity_clean.head()

,type_m,x_start_h,x_min,average_heartrate,max_heartrate,average_cadence
0,Hike,16.8,42.1,127.2,161.0,42.3
1,Hike,17.1,20.1,114.8,148.0,43.0
2,Hike,15.7,62.1,118.6,156.0,43.5
3,Hike,15.3,42.1,125.4,156.0,46.7
4,Hike,16.0,52.1,137.4,154.0,44.7


In [15]:
data2 = activity_clean.dropna()
data2.head()

,type_m,x_start_h,x_min,average_heartrate,max_heartrate,average_cadence
0,Hike,16.8,42.1,127.2,161.0,42.3
1,Hike,17.1,20.1,114.8,148.0,43.0
2,Hike,15.7,62.1,118.6,156.0,43.5
3,Hike,15.3,42.1,125.4,156.0,46.7
4,Hike,16.0,52.1,137.4,154.0,44.7


In [16]:
y = data2["type_m"]
target_names = ["Hike", "Elliptical", "Run"]
X = data2.drop("type_m", axis=1)
X.head()

,x_start_h,x_min,average_heartrate,max_heartrate,average_cadence
0,16.8,42.1,127.2,161.0,42.3
1,17.1,20.1,114.8,148.0,43.0
2,15.7,62.1,118.6,156.0,43.5
3,15.3,42.1,125.4,156.0,46.7
4,16.0,52.1,137.4,154.0,44.7


In [17]:
print(X.shape, y.shape)

(408, 5) (408,)


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [20]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [22]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=5))
model.add(Dense(units=3, activation='softmax'))

In [25]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               600       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 903
Trainable params: 903
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
10/10 - 0s - loss: 1.0668 - accuracy: 0.1765
Epoch 2/60
10/10 - 0s - loss: 0.9261 - accuracy: 0.7941
Epoch 3/60
10/10 - 0s - loss: 0.8009 - accuracy: 0.8039
Epoch 4/60
10/10 - 0s - loss: 0.6975 - accuracy: 0.8039
Epoch 5/60
10/10 - 0s - loss: 0.6250 - accuracy: 0.8039
Epoch 6/60
10/10 - 0s - loss: 0.5794 - accuracy: 0.8039
Epoch 7/60
10/10 - 0s - loss: 0.5542 - accuracy: 0.8039
Epoch 8/60
10/10 - 0s - loss: 0.5407 - accuracy: 0.8039
Epoch 9/60
10/10 - 0s - loss: 0.5294 - accuracy: 0.8039
Epoch 10/60
10/10 - 0s - loss: 0.5186 - accuracy: 0.8039
Epoch 11/60
10/10 - 0s - loss: 0.5087 - accuracy: 0.8039
Epoch 12/60
10/10 - 0s - loss: 0.4984 - accuracy: 0.8039
Epoch 13/60
10/10 - 0s - loss: 0.4897 - accuracy: 0.8039
Epoch 14/60
10/10 - 0s - loss: 0.4793 - accuracy: 0.8039
Epoch 15/60
10/10 - 0s - loss: 0.4702 - accuracy: 0.8039
Epoch 16/60
10/10 - 0s - loss: 0.4606 - accuracy: 0.8039
Epoch 17/60
10/10 - 0s - loss: 0.4511 - accuracy: 0.8039
Epoch 18/60
10/10 - 0s - loss: 0.4414 - 

In [28]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.2910 - accuracy: 0.9314
Normal Neural Network - Loss: 0.29095304012298584, Accuracy: 0.9313725233078003


In [29]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [36]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['Hike' 'Run' 'Run' 'Run' 'Run']
Actual Labels: ['Run', 'Run', 'Run', 'Run', 'Run']


In [39]:
# Calculate classification report
from sklearn.metrics import classification_report
encoded_predictions2 = model.predict_classes(X_test_scaled)
prediction_labels2 = label_encoder.inverse_transform(encoded_predictions2)
prediction_labels2

array(['Hike', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Hike',
       'Run', 'Elliptical', 'Elliptical', 'Run', 'Run', 'Run', 'Run',
       'Elliptical', 'Hike', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run',
       'Elliptical', 'Run', 'Run', 'Run', 'Run', 'Run', 'Elliptical',
       'Hike', 'Run', 'Run', 'Hike', 'Run', 'Run', 'Run', 'Run', 'Run',
       'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run',
       'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Hike', 'Run',
       'Run', 'Run', 'Elliptical', 'Run', 'Run', 'Run', 'Run', 'Hike',
       'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Elliptical',
       'Elliptical', 'Run', 'Hike', 'Elliptical', 'Run', 'Run', 'Run',
       'Run', 'Run', 'Run', 'Hike', 'Run', 'Elliptical', 'Run', 'Run',
       'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run', 'Run',
       'Run', 'Run', 'Run', 'Run'], dtype=object)

In [54]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction_labels2,
                            target_names=target_names))

              precision    recall  f1-score   support

        Hike       0.80      0.67      0.73        12
  Elliptical       0.78      0.78      0.78         9
         Run       0.96      0.99      0.98        81

    accuracy                           0.93       102
   macro avg       0.85      0.81      0.83       102
weighted avg       0.93      0.93      0.93       102



In [45]:
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
cm = confusion_matrix(y_test,prediction_labels2)
cm

array([[ 8,  1,  3],
       [ 2,  7,  0],
       [ 0,  1, 80]], dtype=int64)

In [50]:
plot_confusion_matrix(model,X_test_scaled,y_test)

ValueError: plot_confusion_matrix only supports classifiers